# Final Project Submission
Please fill out:

-  Student name: Steven Rosa
-  Student pace: part time
-  Scheduled project review date/time: Mon 4 Feb 2019 2pm EST
-  Instructor name: Jeff Herman
-  Blog post URL:

# Northwind Data Analysis

<a id="top"></a>

# Contents
- [Import libraries](#importlibraries)
- [Write functions](#writefunctions)
- [Exploratory data analysis](#eda)

<a id="importlibraries"></a>

# Import libraries
[(Back to top)](#top)

In [ ]:
#Import libraries
import sqlalchemy #ORM
from sqlalchemy import create_engine #To connect to the database
from sqlalchemy.orm import Session, sessionmaker #To interact with the database as object
from sqlalchemy import inspect #To look at tables and column names
from sqlalchemy import MetaData #to reflect the engine object 
from sqlalchemy.ext.automap import automap_base # to create mappings from the reflection
import pandas as pd #To read in parts of the database as DataFrames

import matplotlib.pyplot as plt #For creating visualizations
from scipy.stats import norm #For plotting probablity density functions
import seaborn as sns #For plotting pdfs over histograms
import numpy as np #For mathematical operations and arrays

<a id="writefunctions"></a>

# Write functions
[(Back to top)](#top)

In [ ]:
#Build a sampling distribution of the data 
#in order to get it more "normal"
def sampling_dist(data, dist=50, n=30):

    sample_dist = []
   
    while len(sample_dist) != dist: #Build a sample distribution
        sample = []
        while len(sample) != n:
            x = np.random.choice(data) #Get a value from the data with replacement
            sample.append(x) #Add the value to the sample
        sample_mean = sum(sample) / len(sample) #Get the mean of the sample
        sample_dist.append(sample_mean) #Add the mean to the sampling distribution
    
    return sample_dist

<a id="eda"></a>

# Exploratory data analysis
[(Back to top)](#top)

In [ ]:
#Instantiate a session of the ORM
#Connect to the sqlite database file
engine = create_engine('sqlite:///Northwind_small.sqlite', echo = True)
#Bind a session factory to the engine
Session = sessionmaker(bind=engine)
#Instantiate a session
session = Session()

In [ ]:
#An object to get a first look at the database
inspector = inspect(engine)

In [ ]:
#Make a dict of list of names of all the columns in all the tables
#for easy reference

#Get list of all table names
table_names = inspector.get_table_names()

tables_columns = dict()
#For each table name
for table in table_names:
    column_names = []
    #for each column in the table, get just the name
    for column in inspector.get_columns(table):
        column_names.append(column['name'])
    tables_columns[table] = column_names

In [ ]:
tables_columns['OrderDetail']

In [ ]:
#Pull raw results from the database
#conn = engine.connect()
#results = conn.execute('''SELECT *
#                            FROM Category
#                        ;''')
#print(results.fetchall())

In [ ]:
###conn = engine.connect()
#results = conn.execute("SELECT * FROM [Order];")
#df = pd.DataFrame(results.fetchall())
#df.head()

In [ ]:
#results = conn.execute('''SELECT *
#                            FROM OrderDetail
#                        ;''')
#df = pd.DataFrame(results.fetchall())
#df.head()

In [ ]:
#df = pd.read_sql_query('''SELECT * 
#                           FROM [Order]
#                       ;''', engine)
#df.head()

In [ ]:
#df = pd.read_sql_query("SELECT * FROM [Order] WHERE CUSTOMERId = 'VINET'", engine)
#df.head()

In [ ]:
df = pd.read_sql_query("""SELECT OrderDetail.*, [Order].CustomerId 
                            FROM OrderDetail 
                            INNER JOIN [Order] on OrderDetail.OrderId = [Order].Id
                            WHERE OrderDetail.Discount != 0.0
                        """, engine)
df.head()

In [ ]:
#WHERE OrderDetail.Discount != 0.0

In [ ]:
#Look at all orders with and without discounts
df = pd.read_sql_query("""SELECT OrderDetail.*, [Order].CustomerId
                            FROM OrderDetail 
                            INNER JOIN [Order] on OrderDetail.OrderId = [Order].Id
                            ORDER BY [Order].CustomerId DESC, [Order].Id DESC, OrderDetail.Discount DESC
                        """, engine)


In [ ]:
#Look at all orders with and without discounts
df.head()

In [ ]:
#Look at number of orders per cust id
df = pd.read_sql_query("""SELECT [Order].CustomerId, Count(*) orders
                            FROM OrderDetail 
                            INNER JOIN [Order] on OrderDetail.OrderId = [Order].Id
                            GROUP BY [Order].CustomerId
                            ORDER BY [Order].CustomerId ASC
                        """, engine)

In [ ]:
#Look at number of orders per cust id
df.head()

In [ ]:
#Look at number of orders per customer w/o discount
df_wo_discount = pd.read_sql_query("""SELECT [Order].CustomerId, Count(*) orders
                            FROM OrderDetail 
                            INNER JOIN [Order] on OrderDetail.OrderId = [Order].Id
                            WHERE OrderDetail.Discount == 0.0
                            GROUP BY [Order].CustomerId
                            ORDER BY [Order].CustomerId ASC
                        """, engine)

#Look at number of orders per customer w/ discount
df_w_discount = pd.read_sql_query("""SELECT [Order].CustomerId, Count(*) orders
                            FROM OrderDetail 
                            INNER JOIN [Order] on OrderDetail.OrderId = [Order].Id
                            WHERE OrderDetail.Discount != 0.0
                            GROUP BY [Order].CustomerId
                            ORDER BY [Order].CustomerId ASC
                        """, engine)

In [ ]:
df_wo_discount.head()

In [ ]:
df_w_discount.head()

In [ ]:
#Look at the levels of discount

df = pd.read_sql_query("""SELECT OrderDetail.*
                            FROM OrderDetail 
                        """, engine)
df['Discount'].value_counts()

In [ ]:
#Look at number of orders w and wo discount per cust id
###Insert soup for Emily with no bacon
df = pd.read_sql_query('''SELECT [Order].CustomerId,
                            SUM(
                                CASE WHEN OrderDetail.Discount == 0.0 
                                THEN 1 ELSE 0 END
                                ) no_disc,
                            SUM(
                                CASE WHEN OrderDetail.Discount != 0.0 
                                THEN 1 ELSE 0 END
                                ) all_discs,
                            SUM(
                                CASE WHEN OrderDetail.Discount < 0.06 
                                AND OrderDetail.Discount > 0.0
                                THEN 1 ELSE 0 END
                                ) disc06,
                            SUM(
                                CASE WHEN OrderDetail.Discount == 0.1 
                                THEN 1 ELSE 0 END
                                ) disc10,
                            SUM(
                                CASE WHEN OrderDetail.Discount == 0.15 
                                THEN 1 ELSE 0 END
                                ) disc15,
                            SUM(
                                CASE WHEN OrderDetail.Discount == 0.2 
                                THEN 1 ELSE 0 END
                                ) disc20,
                            SUM(
                                CASE WHEN OrderDetail.Discount == 0.25 
                                THEN 1 ELSE 0 END
                                ) disc25,
                            Count(*) total_orders
                            FROM OrderDetail 
                            INNER JOIN [Order] on OrderDetail.OrderId = [Order].Id
                            GROUP BY [Order].CustomerId
                            ORDER BY [Order].CustomerId ASC
                        ''', engine)



In [ ]:
df.head()

In [ ]:
#See what one order looks like 
df = pd.read_sql_query('''SELECT *
                            FROM OrderDetail
                            LIMIT 1
                        ''', engine)

In [ ]:
#See what one order looks like
df

In [ ]:
#See what orders look like with total money spent
df = pd.read_sql_query('''SELECT *,
                                (od.UnitPrice * od.Quantity) total_price
                            FROM OrderDetail od
                        ''', engine)

In [ ]:
df.head()


In [ ]:
#Compare quantities ordered with and without discounts

discs_ords = pd.read_sql_query('''SELECT [Order].CustomerId,
                            SUM(
                                CASE WHEN OrderDetail.Discount == 0.0 
                                THEN 1 ELSE 0 END
                                ) no_disc,
                            SUM(
                                CASE WHEN OrderDetail.Discount == 0.0
                                THEN OrderDetail.Quantity ELSE 0 END
                                ) quant_no_disc,
                            SUM(
                                CASE WHEN OrderDetail.Discount != 0.0 
                                THEN 1 ELSE 0 END
                                ) all_discs,
                            SUM(
                                CASE WHEN OrderDetail.Discount != 0.0
                                THEN OrderDetail.Quantity ELSE 0 END
                                ) quant_all_discs,
                            SUM(
                                CASE WHEN OrderDetail.Discount < 0.06 
                                AND OrderDetail.Discount > 0.0
                                THEN 1 ELSE 0 END
                                ) disc06,                            
                            SUM(
                                CASE WHEN OrderDetail.Discount < 0.06
                                AND OrderDetail.Discount > 0.0
                                THEN OrderDetail.Quantity ELSE 0 END
                                ) quant_disc06,
                            SUM(
                                CASE WHEN OrderDetail.Discount == 0.1 
                                THEN 1 ELSE 0 END
                                ) disc10,
                            SUM(
                                CASE WHEN OrderDetail.Discount == 0.1
                                THEN OrderDetail.Quantity ELSE 0 END
                                ) quant_disc10,                            
                            SUM(
                                CASE WHEN OrderDetail.Discount == 0.15 
                                THEN 1 ELSE 0 END
                                ) disc15,
                            SUM(
                                CASE WHEN OrderDetail.Discount == 0.15
                                THEN OrderDetail.Quantity ELSE 0 END
                                ) quant_disc15,                            
                            SUM(
                                CASE WHEN OrderDetail.Discount == 0.2 
                                THEN 1 ELSE 0 END
                                ) disc20,
                            SUM(
                                CASE WHEN OrderDetail.Discount == 0.2
                                THEN OrderDetail.Quantity ELSE 0 END
                                ) quant_disc20,                                
                            SUM(
                                CASE WHEN OrderDetail.Discount == 0.25 
                                THEN 1 ELSE 0 END
                                ) disc25,
                            SUM(
                                CASE WHEN OrderDetail.Discount == 0.25
                                THEN OrderDetail.Quantity ELSE 0 END
                                ) quant_disc25,
                            Count(*) total_orders,
                            SUM(OrderDetail.Quantity) total_quantities
                            FROM OrderDetail 
                            INNER JOIN [Order] on OrderDetail.OrderId = [Order].Id
                            GROUP BY [Order].CustomerId
                            ORDER BY [Order].CustomerId ASC
                        ''', engine)


In [ ]:
#Compare quantities ordered with and without discounts
discs_ords.head()

In [ ]:
#Sanity check for quantities
#w/o discount. ALL GOOD
#df_wo_discount = pd.read_sql_query('''SELECT [Order].CustomerId, 
#                                        SUM(
#                                            CASE WHEN OrderDetail.Discount == 0.0
#                                            THEN OrderDetail.Quantity ELSE 0 END
#                                            ) quants_nodisc
#                                        FROM OrderDetail
#                                        INNER JOIN [Order] on OrderDetail.OrderId = [Order].Id
#                                        GROUP BY [Order].CustomerId
#                                        ORDER BY [Order].CustomerId ASC
#                                    ''', engine)

#w/ discount
#df_w_discount = pd.read_sql_query('''SELECT [Order].CustomerId,
#                                        SUM(
#                                            CASE WHEN OrderDetail.Discount != 0.0
#                                            THEN OrderDetail.Quantity ELSE 0 END
#                                            ) quants_disc
#                                        FROM OrderDetail 
#                                        INNER JOIN [Order] on OrderDetail.OrderId = [Order].Id
#                                        WHERE OrderDetail.Discount != 0.0
#                                        GROUP BY [Order].CustomerId
#                                        ORDER BY [Order].CustomerId ASC
#                                ''', engine)

In [ ]:
#df_wo_discount.head()

In [ ]:
#df_w_discount.head()

In [ ]:
discs_ords.shape

In [ ]:
discs_ords['quant_no_disc'].mean()

In [ ]:
discs_ords['quant_all_discs'].mean()

In [ ]:
discs_ords['quant_no_disc'].sum()

In [ ]:
discs_ords['quant_all_discs'].sum()

In [ ]:
#Not sure whether grouping is affecting the quantity sums per order. 
#i.e. is it adding up the quantities for every record assoc with an order id or just for one?
#Look at number of orders per cust id

alfki = pd.read_sql_query("""SELECT [Order].CustomerId, OrderDetail.*
                            FROM OrderDetail 
                            
                            INNER JOIN [Order] on OrderDetail.OrderId = [Order].Id
                            WHERE [Order].CustomerId == 'ALFKI'
                        """, engine)

In [ ]:
#PROVEN: sum is of quantities for every record not the group
alfki['Quantity'].sum()

In [ ]:
len(discs_ords['quant_no_disc'])

In [ ]:
#Compare quantitites ordered per customer w and wo discount
fig = plt.figure(figsize=(8,6))
plt.scatter(range(len(discs_ords['quant_all_discs'])), discs_ords['quant_all_discs'], color='red')
plt.scatter(range(len(discs_ords['quant_no_disc'])), discs_ords['quant_no_disc'], color='blue')

In [ ]:
#Spread of raw data looks extremely close between the two. Shoudl look at hists and pdfs of each

In [ ]:
plt.hist(discs_ords['quant_all_discs'], bins=20)

In [ ]:
plt.hist(discs_ords['quant_no_disc'], bins=20)

In [ ]:
#Again, the two distributions look very close, both with greatest density around the lower values
#At this point I can predict that there won't be a significant difference between the two

In [ ]:
plt.hist(discs_ords['quant_no_disc'], normed=True, bins=20)

In [ ]:
#Show the distribution with a pdf to check for normality
sns.distplot(discs_ords['quant_no_disc'])

In [ ]:
#Get the data to look more "normal"
no_disc_sampling = np.array(sampling_dist(np.array(discs_ords['quant_no_disc']), 100))

In [ ]:
no_disc_sampling.mean(), discs_ords['quant_no_disc'].mean()

In [ ]:
sns.distplot(no_disc_sampling)

In [ ]:
#A sampling distribution of the quantities order without discounts looks normal-er

In [ ]:
disc_sampling = np.array(sampling_dist(np.array(discs_ords['quant_all_discs']), 100))
disc_sampling.mean(), discs_ords['quant_all_discs'].mean()

In [ ]:
sns.distplot(disc_sampling)

In [ ]:
####EDA
#Look at how many customers each company has
df = pd.read_sql_query("""SELECT Customer.Id, Customer.CompanyName
                            FROM Customer
                            ORDER BY Customer.CompanyName
                        """, engine)

In [ ]:
#Every company has only one customer. Orders can therefore be counted by customer or by company.
sum(df['CompanyName'].value_counts() > 1)

In [ ]:
# JUST AN EXAMPLE from the labs
#"""SELECT o.ID, c.CompanyName, Count(*) num_orders 
#    FROM [Order] o
#    INNER JOIN Customer c on o.CustomerID = c.ID 
#    GROUP BY c.CompanyName 
#    ORDER BY num_orders DESC"""

#df = pd.read_sql_query("""SELECT o.ID, c.CompanyName, Count(*) num_orders FROM [Order] \
#o INNER JOIN Customer c on o.CustomerID = c.ID GROUP BY c.CompanyName ORDER BY num_orders DESC""", engine)
#df.head()

In [ ]:
#######################################
#AUTOMAPPER BITS
#Instantiate metadata to reflect the engine
#metadata = MetaData()
#metadata.reflect(engine)

In [ ]:
#Create mappings to treat the database elements as OOP objects
Base = automap_base(metadata=metadata)
Base.prepare()

In [ ]:
#['Category', 'Customer', 'CustomerCustomerDemo', 'CustomerDemographic', 
#'Employee', 'EmployeeTerritory', 'Order', 'OrderDetail', 'Product', 'Region', 'Shipper', 'Supplier', 'Territory']

In [ ]:
#Create an object for each table
Category = Base.classes.Category
Customer = Base.classes.Customer
CustomerCustomerDemo = Base.classes.CustomerCustomerDemo
CustomerDemographic = Base.classes.CustomerDemographic
Employee = Base.classes.Employee
EmployeeTerritory = Base.classes.EmployeeTerritory
Order = Base.classes.Order
OrderDetail = Base.classes.OrderDetail
Product = Base.classes.Product
Region = Base.classes.Region
Shipper = Base.classes.Shipper
Supplier = Base.classes.Supplier
Territory = Base.classes.Territory

In [ ]:
categories = session.query(Category).all()

for cat in categories:
    print(cat.CategoryName, cat.Description)

In [ ]:
for instance in session.query(Category):
    print(instance.Description)

In [ ]:
countries = []
for instance in session.query(Customer):
    countries.append(instance.Country)

In [ ]:
countries_df = pd.DataFrame(countries)
countries_df[0].value_counts()

In [ ]:
regions = []
for instance in session.query(Customer):
    regions.append(instance.Region)

regions_df = pd.DataFrame(regions)
regions_df[0].value_counts()

In [ ]:
names = []
for instance in session.query(Customer):
    names.append(instance.ContactName)
    
names_df = pd.DataFrame(names)
names_df[0].value_counts()

In [ ]:
len(set(names))

In [ ]:
session.query(Customer).column_descriptions

In [ ]:
result = session.execute('''SELECT *
                            FROM Customer
                        ;''')

In [ ]:
customer_df = pd.DataFrame(result.fetchall())

In [ ]:
customer_df.shape

## Hypothesis 1
Do discounts have a statistically significant effect on the number of products customers order? If so, at what level(s) of discount?

product (quantity) ordered by customers, no discount

product (quantity) order by customers, wiht discount

H0: Discount does not have an effect on the number of products ordered by a customer. Control = Treatment

H1: Discount has an effect on the number of products ordered by a customer. Control < or > Treament